In [1]:
import pandas as pd
import numpy as np
import math as m
import DataHandling

In [213]:
roadId = input('Please write the Road ID: ')

Please write the Road ID: N1


In [201]:
roads = DataHandling.readRoads(roadId)
bridges = DataHandling.readBridges(roadId)
widths = DataHandling.readWidths(roadId)
traffic = DataHandling.readTraffic(roadId)
roadnames = DataHandling.readBridges('').drop_duplicates('road', keep = 'first')['road'].tolist()

In [202]:
def formatWidthData(dataframe = widths):
    dataframe = dataframe.drop('width', 1)
    dataframe = dataframe.drop('roadId', 1)
    dataframe = dataframe.sort_values(by=['road','startChainage']).reset_index()
    dataframe = dataframe.drop('index', 1)
    return dataframe

In [203]:
def formatBridgeData(dataframe = bridges):
    dataframenoduplicates = dataframe.drop_duplicates('LRPName', keep = 'first')
    dataframemodified = dataframenoduplicates.ix[:,:10]
    dataframemodified = dataframemodified.drop('km', 1)
    dataframemodified = dataframemodified.drop('structureNr', 1)
    dataframemodified = dataframemodified.drop('type', 1)
    dataframemodified = dataframemodified.drop('roadName', 1)
    dataframemodified = dataframemodified.drop('length', 1)
    dataframemodified = dataframemodified.drop('name', 1)
    dataframemodified = dataframemodified.sort_values(by=['road','chainage']).reset_index()
    dataframemodified = dataframemodified.drop('index', 1)
    return dataframemodified

In [204]:
def formatTrafficData(dataframe = traffic, sideofroad = 'both'):
    dataframe = dataframe.drop('name',1)
    dataframe = dataframe.drop('start_lrp',1)
    dataframe = dataframe.drop('start_offset',1)
    dataframe = dataframe.drop('end_lrp',1)
    dataframe = dataframe.drop('end_offset',1)
    dataframe = dataframe.drop('length',1)
    dataframe = dataframe.reset_index()
    dataframe = dataframe.drop('index',1)
    dataframe.columns = dataframe.columns.str.replace('start_chainage','startChainage')
    dataframe.columns = dataframe.columns.str.replace('end_chainage','endChainage')
    dataframe = dataframe.sort_values(by = ['road','startChainage','linkNo'] ).reset_index()
    dataframe = dataframe.drop('index', 1)
    if sideofroad == 'both':
        dataframe = dataframe.groupby(['road','startChainage'],as_index=False).sum()
    return dataframe

In [205]:
def mergeFiles(trafficData=formatTrafficData(), bridgeData = formatBridgeData(), widthData = formatWidthData(), roadside = 'both'):
    indexesMergedData = widthData['startChainage'].searchsorted(bridgeData['chainage'], side = 'right')
    indexesMergedData =indexesMergedData.tolist()
    addedNrLanes = bridgeData
    list_a = []
    for i in indexesMergedData:
        list_a.append(widthData['nrLanes'][int(i)-1])
    addedNrLanes['nrLanes']=list_a
    bridgeData = addedNrLanes
    addedTrafficData = bridgeData
    for title in trafficData.ix[:,3:].columns:
        indexesMergedData = trafficData['startChainage'].searchsorted(bridgeData['chainage'], side = 'right')
        indexesMergedData =indexesMergedData.tolist()
        list_b=[]
        for i in indexesMergedData:
            list_b.append(trafficData[title][int(i)-1])
        addedTrafficData[title]=list_b
    return addedTrafficData

In [206]:
def calculateLaneTrafficDensity(dataframe = mergeFiles()):
    for title in dataframe.ix[:,5:].columns:
        dataframe[title] = dataframe[title]/dataframe['nrLanes']
    return dataframe

In [207]:
#Truck capacity is max capacity of type of trucks in tonnes
def calculateVulnerability(dataframe = calculateLaneTrafficDensity(), truckcapacity = [15,10,5], socialcarcapacity = [45,30,15,6,4,2,2,1,2,1], conditionlikelihood = [0.05,0.10,0.15,0.20]):
    vulbasematrix = dataframe.ix[:,:4]
    vulbasematrix.loc[vulbasematrix.condition == 'A', 'BridgeFailureLikelihood'] = conditionlikelihood[0]
    vulbasematrix.loc[vulbasematrix.condition == 'B', 'BridgeFailureLikelihood'] = conditionlikelihood[1]
    vulbasematrix.loc[vulbasematrix.condition == 'C', 'BridgeFailureLikelihood'] = conditionlikelihood[2]
    vulbasematrix.loc[vulbasematrix.condition == 'D', 'BridgeFailureLikelihood'] = conditionlikelihood[3]
    cargotitles = ['heavyTruck', 'mediumTruck', 'smallTruck']
    socialtitles = ['largeBus','mediumBus','microBus','utility','car','autoRickshaw','motorcycle','bicycle','cycleRickshaw','cart']
    vulbasematrix['TrafficEconomicVulnerability'] = 0
    for i in range(len(cargotitles)):
        vulbasematrix['TrafficEconomicVulnerability'] =  vulbasematrix['TrafficEconomicVulnerability']+dataframe[cargotitles[i]]*truckcapacity[i]
    vulbasematrix['TotalEconomicVulnerability'] = vulbasematrix['TrafficEconomicVulnerability']*vulbasematrix['BridgeFailureLikelihood']
    vulbasematrix['TrafficSocialVulnerability'] = 0
    for i in range(len(socialtitles)):
        vulbasematrix['TrafficSocialVulnerability'] =  vulbasematrix['TrafficSocialVulnerability']+dataframe[socialtitles[i]]*socialcarcapacity[i]
    vulbasematrix['TotalSocialVulnerability'] = vulbasematrix['TrafficSocialVulnerability']*vulbasematrix['BridgeFailureLikelihood']
    return vulbasematrix

In [208]:
def printRoadResults(dataframe = calculateVulnerability(),  roadId = roadId, root = 'vulnerability'):
    dataframe.to_csv(root+roadId+'.csv')


In [235]:
def calculatevulnerabilitysegment(dataframe = calculateVulnerability(), groupLevel=1000.0):
    result = dataframe.groupby(['road',np.floor(dataframe.index/groupLevel)], as_index = False)
    bridgereliability = result.apply(lambda x: np.product(1-x['BridgeFailureLikelihood'])).reset_index(drop=True)
    lista = []
    for i in bridgereliability:
        lista.append(i.max())
    result = result.max()
    result['BridgeFailureLikelihood'] = 1-pd.DataFrame(lista)
    result['TotalSocialVulnerability'] = result['TrafficSocialVulnerability']*result['BridgeFailureLikelihood']
    result['TotalEconomicVulnerability'] = result['TrafficEconomicVulnerability']*result['BridgeFailureLikelihood']
    return  result

In [236]:
calculatevulnerabilitysegment()

,road,LRPName,condition,chainage,BridgeFailureLikelihood,TrafficEconomicVulnerability,TotalEconomicVulnerability,TrafficSocialVulnerability,TotalSocialVulnerability
0,N1,LRP456c,D,460.113,1.0,2260.000000,2260.000000,8813.000000,8813.000000
1,N102,LRPSc,A,73.118,1.0,1570.000000,1570.000000,7035.000000,7035.000000
2,N104,LRP044d,A,44.840,1.0,1570.000000,1570.000000,7035.000000,7035.000000
3,N105,LRP041c,A,40.635,1.0,1570.000000,1570.000000,7035.000000,7035.000000
4,N106,LRPSb,B,54.662,1.0,1570.000000,1570.000000,7035.000000,7035.000000
5,N108,LRP021e,A,21.689,1.0,1570.000000,1570.000000,7035.000000,7035.000000
6,N110,LRPsd,A,6.400,1.0,1130.000000,1130.000000,9680.000000,9680.000000
7,N2,LRP285c,C,284.640,1.0,1570.000000,1570.000000,7035.000000,7035.000000
8,N207,LRP061d,A,61.306,1.0,1570.000000,1570.000000,7035.000000,7035.000000
9,N208,LRP058d,A,59.180,1.0,1735.000000,1735.000000,9395.000000,9395.000000
